In [1]:
import os
import pickle
import random
import time
import numpy as np

from tensorflow.python.client import device_lib
from tensorflow.python.keras.layers import RepeatVector, TimeDistributed, Bidirectional

from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.image import load_img, img_to_array

from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add

# Расположение файлов

In [2]:
curr_folder = "D:/YandexDisk/datasets/"
end_dir = "D:/datasets/flickr-images-12k"

path_tokenizer = curr_folder + "ru-12k-tokenizer-train.pkl"
path_train_dict = curr_folder + "captions-ru-12k-train.pkl"
path_val_dict = curr_folder + "captions-ru-12k-val.pkl"

path_features_vgg16 = curr_folder + "ru-12k-features.pkl"
path_features_inception = "D:/features-inception"

# Проверка видеокарты

In [3]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13331041716526494644
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3129068339
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5897821161333642525
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


# Загрузка данных

Мы собираемся обучить данные по всем фотографиям и подписям в обучающем наборе данных. Во время обучения мы будем отслеживать производительность модели в наборе данных разработки и использовать эту производительность, чтобы решить, когда сохранять модели в файл.

Модель, которую мы разработаем, будет генерировать подпись к фотографии, и подпись будет генерироваться по одному слову за раз.

Последовательность ранее сгенерированных слов будет предоставлена в качестве входных данных. Поэтому нам понадобится "первое слово", чтобы начать процесс генерации, и "последнее слово", чтобы сигнализировать об окончании подписи. Для этой цели мы будем использовать строки "startseq" и "endseq". Эти маркеры добавляются к загруженным описаниям по мере их загрузки. Важно сделать это сейчас, прежде чем мы закодируем текст, чтобы токены также были закодированы правильно.

In [4]:
def image_names_set(data):
    vals = set()

    for idx in data.index:
        vals.add(data.iat[idx, 0][:-4])

    return vals

def load_image_features(filename, data):
    all_features = pickle.load(open(filename, 'rb'))
    features = {k: all_features[k] for k in data}

    return features

# Закодировать знаки в числа

Текст описания необходимо будет закодировать в числа, прежде чем его можно будет представить модели в качестве входных данных или сравнить с предсказаниями модели.

Первым шагом в кодировании данных является создание согласованного сопоставления слов с уникальными целочисленными значениями. Keras предоставляет класс Tokenizer, который может изучить это сопоставление из загруженных данных описания.

Каждое описание будет разделено на слова. Модель будет предоставлена одним словом и фотографией и сгенерирует следующее слово. Затем первые два слова описания будут предоставлены модели в качестве входных данных вместе с изображением для создания следующего слова. Именно так будет обучаться модель.

In [5]:
def to_lines(data):
    all_vals = list()
    for key in data.keys():
        [all_vals.append(d) for d in data[key]]

    return all_vals

def create_tokenizer(data):
    lines = to_lines(data)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)

    return tokenizer

def find_max_words(data):
    lines = to_lines(data)
    return max(len(l.split()) for l in lines)

# Создание последовательности

Приведенная ниже функция с именем create_sequences(), учитывая токенизатор, максимальную длину последовательности и словарь всех описаний и фотографий, преобразует данные в пары ввода-вывода данных для обучения модели.

В модели есть два входных массива: один для признаков фотографии и один для закодированного текста. Существует один вывод для модели, который представляет собой закодированное следующее слово в текстовой последовательности.

Входной текст кодируется в виде целых чисел, которые будут передаваться на слой встраивания слов. Признаки изображения будут передаваться непосредственно в другую часть модели. Модель выдаст прогноз, который будет представлять собой распределение вероятностей по всем словам в словаре.

Таким образом, выходные данные будут представлять собой однократно закодированную версию каждого слова, представляющую идеализированное распределение вероятностей со значениями 0 во всех позициях слов, кроме фактической позиции слова, которая имеет значение 1.

In [6]:
def create_sequences(tokenizer, max_words, captions_list, image_name):
    X_image, X_text, y_word = list(), list(), list()
    vocab_size = len(tokenizer.word_index) + 1

    for caption in captions_list:
        seq = tokenizer.texts_to_sequences([caption])[0]

        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_words)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

            X_image.append(image_name)
            X_text.append(in_seq)
            y_word.append(out_seq)

    return X_image, X_text, y_word

# Генератор данных

Генератор данных будет выдавать данные на одно изображении в каждой партии. Это будут все последовательности, сгенерированные для изображения и её набора описаний.

Функция data_generator() будет генератором данных и будет принимать загруженные текстовые описания, признаки изображений, токенизатор и максимальную длину.8 ГБ оперативной памяти должно быть более чем достаточно.

Вы можете видеть, что мы вызываем функцию create_sequence (), чтобы создать пакет данных для одного изображения, а не для всего набора данных. Это означает, что мы должны обновить функцию create_sequences (), чтобы удалить “итерацию по всем описаниям” для цикла.

Генератор данных, предназначен для использования в вызове model.fit_generator().

Обратите внимание, что это очень простой генератор данных. Большая экономия памяти, которую он предлагает, заключается в том, чтобы не иметь развернутых последовательностей обучающих и тестовых данных в памяти до подгонки модели, чтобы эти образцы (например, результаты create_sequences()) создавались по мере необходимости для каждого изображения.

Некоторые нестандартные идеи для дальнейшего совершенствования этого генератора данных включают в себя:
– Рандомизируйте порядок фотографий каждой эпохи.
– Работайте со списком идентификаторов изображений и загружайте текст и данные изображений по мере необходимости, чтобы ещё больше сократить объём памяти.
– Получите более одного изображения в партии.

In [7]:
def data_generator(tokenizer, max_words, data, images, batch_size, random_seed):
    count = 0
    random.seed(random_seed)

    img_names = list(data.keys())
    assert batch_size <= len(img_names), 'batch size must be less than or equal to {}'.format(len(img_names))

    while True:
        input_img_batch, input_seq_batch, output_word_batch = list(), list(), list()

        if count >= len(img_names):
            count = 0
        start_i = count
        end_i = min(len(img_names), count + batch_size)

        for i in range(start_i, end_i):
            curr_img = img_names[i]
            image = images[curr_img][0]
            captions_list = data[curr_img]
            random.shuffle(captions_list)

            input_img, input_seq, output_word = create_sequences(tokenizer, max_words, captions_list, image)

            for j in range(len(input_img)):
                input_img_batch.append(input_img[j])
                input_seq_batch.append(input_seq[j])
                output_word_batch.append(output_word[j])

        count = count + batch_size
        yield [np.array(input_img_batch), np.array(input_seq_batch)], np.array(output_word_batch)

# Построение модели

Мы опишем модель в трёх частях:

1 – Извлечение признаков изображения. Это 16-слойная модель VGG, предварительно обученная на наборе данных ImageNet. Мы предварительно обработали изображения с помощью модели VGG (без выходного слоя) и будем использовать извлечённые признаки, предсказанные этой моделью, в качестве входных данных.

2 – Обработка последовательностей. Это слой встраивания слов для обработки ввода текста, за которым следует слой рекуррентной нейронной сети с длительной кратковременной памятью (LSTM).

3 – Расшифровка. (1) и (2) выводят вектор фиксированной длины. Они объединяются вместе и обрабатываются плотным слоем, чтобы сделать окончательный прогноз.

Модель (1) ожидает, что входные признаки изображений будут вектором из 4096 элементов. Они обрабатываются плотным слоем для получения 256-элементного представления изображения.

Модель (2) ожидает входные последовательности с заранее определённой длиной, которые подаются в слой встраивания, использующий маску для игнорирования дополненных значений. За этим следует слой LSTM с 256 единицами памяти.

Обе входные модели создают вектор из 256 элементов. Кроме того, обе входные модели используют регуляризацию в виде 50% отсева (dropout). Это делается для того, чтобы уменьшить переобучение модели на текущем наборе данных, так как эта конфигурация модели обучается очень быстро.

Модель (3) объединяет векторы из обеих входных моделей с помощью операции сложения. Затем этот вектор подаётся на плотный слой из 256 нейронов, а затем на конечный выходной плотный слой, который делает прогноз softmax по всему выходному словарю для следующего слова в последовательности.

In [8]:
def build_rnn(input_size, vocab_size, max_words):
    inputs1 = Input(shape=(input_size,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    inputs2 = Input(shape=(max_words,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    de1 = add([fe2, se3])
    de2 = Dense(256, activation='relu')(de1)
    outputs = Dense(vocab_size, activation='softmax')(de2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [8]:
def build_alt_rnn(input_size, vocab_size, max_words):
    image_input = Input(shape=(input_size,))
    fe1 = Dense(256, activation='relu')(image_input)
    image_model = RepeatVector(max_words)(fe1)

    caption_input = Input(shape=(max_words,))
    # we zero pad inputs to the same length, the zero mask ignores those inputs
    se1 = Embedding(vocab_size, 256, mask_zero=True)(caption_input)
    # since we are going to predict the next word using the previous words
    # (length of previous words changes with every iteration over the caption), we have to set return_sequences = True
    se2 = LSTM(256, return_sequences=True)(se1)
    caption_model = TimeDistributed(Dense(256))(se2)

    # merging the models and creating a softmax classifier
    de1 = add([image_model, caption_model])
    de2 = Bidirectional(LSTM(256, return_sequences=False))(de1)
    final_model = Dense(vocab_size, activation='softmax')(de2)

    model = Model(inputs=[image_input, caption_input], outputs=final_model)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

# Обучение

В этом примере мы отбросим загрузку тестового набора данных и контрольные точки модели и просто сохраним модель после каждой эпохи обучения. Затем мы сможем вернуться и загрузить/оценить каждую сохраненную модель после обучения, чтобы найти ту, которая имеет наименьшие потери.

In [16]:
def load_train_data(train_dict_path, tokenizer_path, features_path):
    with open (train_dict_path, 'rb') as f:
        out_train_dict = pickle.load(f)
    print('кол-во подписей .............. %d' % len(out_train_dict))

    out_train_features = load_image_features(features_path, out_train_dict)

    with open (tokenizer_path, 'rb') as f:
        out_tokenizer = pickle.load(f)
    out_vocab_size = len(out_tokenizer.word_index) + 1
    print('размер словаря ............... %d' % out_vocab_size)

    out_max_words = find_max_words(out_train_dict)
    print('длина предложения в словах ... %d' % out_max_words)

    return out_train_dict, out_tokenizer, out_vocab_size, out_max_words, out_train_features

def train_model(model, train_dict, tokenizer, max_words, train_features, batch_size, epochs_num):
    steps = len(train_dict)/batch_size
    if len(train_dict) % batch_size != 0:
        steps = steps + 1

    start_time = time.time()
    for i in range(epochs_num):
        generator = data_generator(tokenizer, max_words, train_dict, train_features, batch_size, 42)
        model.fit(generator,
                  epochs=1, steps_per_epoch=steps,
                  verbose=1)
        model.save('model-' + str(i) + '.h5')

    time_difference = time.time() - start_time
    minutes = time_difference/60
    print('время обучения в минутах ..... %d' % minutes)

### Набор для обучения

In [12]:
batch_size = 16
epochs_num = 20
train_dict, tokenizer, vocab_size, max_words, train_features = load_train_data(path_train_dict, path_tokenizer, path_features_inception)

кол-во подписей .............. 8262
размер словаря ............... 21391
длина предложения в словах ... 22


# Обучение VGG16

In [16]:
model = build_rnn(4096, vocab_size, max_words)
train_model(model, train_dict, tokenizer, max_words, train_features, batch_size, epochs_num)

517/517 [==============================] - 271s 524ms/step - loss: 2.0093
время обучения в минутах ..... 88


In [12]:
model = build_alt_rnn(4096, vocab_size, max_words)
train_model(model, train_dict, tokenizer, max_words, train_features, batch_size, epochs_num)

517/517 [==============================] - 509s 984ms/step - loss: 1.9104

# Обучение Inception

In [14]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as preprocess_input_i

def extract_features_inception(directory):
    model = InceptionV3(weights="imagenet")
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

    features = dict()
    for name in os.listdir(directory):
        filename = directory + '/' + name

        image = load_img(filename, target_size=(299, 299))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input_i(image)

        feature = model.predict(image, verbose=0)
        image_id = name.split('.')[0]
        features[image_id] = feature

    return features

In [8]:
%%time
features = extract_features_inception(end_dir)
print('выделенные признаки ... %d' % len(features))
pickle.dump(features, open(path_features_inception, 'wb'))

выделенные признаки ... 11804
Wall time: 28min 9s


In [3]:
model = build_rnn(2048, vocab_size, max_words)
train_model(model, train_dict, tokenizer, max_words, train_features, batch_size, epochs_num)

517/517 [==============================] - 331s 640ms/step - loss: 2.0639
время обучения в минутах ..... 99


In [10]:
model = build_alt_rnn(2048, vocab_size, max_words)
train_model(model, train_dict, tokenizer, max_words, train_features, batch_size, epochs_num)

517/517 [==============================] - 523s 1s/step - loss: 1.8748